# Phase 2: Trajectory Prediction with Auxiliary Depth Estimation

# 🧭 Introduction

"""
Welcome to **Phase 2** of the DLAV Projec! 🚗💨

In this phase, you'll work with a more challenging dataset that includes:
- RGB **camera images**
- Ground-truth **depth maps**
- Ground-truth **semantic segmentation** labels

Your goal is still to predict the **future trajectory** of the self-driving car (SDC), but you now have more tools at your disposal! 🎯

Here, we provide an example where **depth estimation** is used as an auxiliary task to improve trajectory prediction.

However, you're **free to explore** other auxiliary tasks (e.g., using semantic labels), different loss functions, data augmentations, or better architectures! 💡

This notebook will walk you through loading the dataset, building a model, training with and without the auxiliary task, and visualizing results.
"""

In [ ]:
# Install gdown to handle Google Drive file download
!pip install -q gdown

import gdown
import zipfile

download_url = f"https://drive.google.com/uc?id=1YkGwaxBKNiYL2nq--cB6WMmYGzRmRKVr"
output_zip = "dlav_train.zip"
gdown.download(download_url, output_zip, quiet=False)  # Downloads the file to your drive
with zipfile.ZipFile(output_zip, 'r') as zip_ref:  # Extracts the downloaded zip file
    zip_ref.extractall(".")

download_url = "https://drive.google.com/uc?id=1wtmT_vH9mMUNOwrNOMFP6WFw6e8rbOdu"
output_zip = "dlav_val.zip"
gdown.download(download_url, output_zip, quiet=False)
with zipfile.ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall(".")

download_url = "https://drive.google.com/uc?id=1G9xGE7s-Ikvvc2-LZTUyuzhWAlNdLTLV"
output_zip = "dlav_test_public.zip"
gdown.download(download_url, output_zip, quiet=False)
with zipfile.ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall(".")

Downloading...
From (original): https://drive.google.com/uc?id=1YkGwaxBKNiYL2nq--cB6WMmYGzRmRKVr
From (redirected): https://drive.google.com/uc?id=1YkGwaxBKNiYL2nq--cB6WMmYGzRmRKVr&confirm=t&uuid=ce3622bc-785a-4b1b-9470-ca5685617e75
To: /content/dlav_train.zip
100%|██████████| 439M/439M [00:07<00:00, 57.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1wtmT_vH9mMUNOwrNOMFP6WFw6e8rbOdu
From (redirected): https://drive.google.com/uc?id=1wtmT_vH9mMUNOwrNOMFP6WFw6e8rbOdu&confirm=t&uuid=0f8f6018-ac89-4392-88f4-a9ba9c608d2c
To: /content/dlav_val.zip
100%|██████████| 87.8M/87.8M [00:01<00:00, 71.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1G9xGE7s-Ikvvc2-LZTUyuzhWAlNdLTLV
From (redirected): https://drive.google.com/uc?id=1G9xGE7s-Ikvvc2-LZTUyuzhWAlNdLTLV&confirm=t&uuid=c8f8b8ab-9f04-422d-b00a-95e00792340e
To: /content/dlav_test_public.zip
100%|██████████| 86.6M/86.6M [00:01<00:00, 86.3MB/s]


## 📂 The Dataset

We are now working with a richer dataset that includes not just images and trajectories,
but also **depth maps** (and semantic segmentation labels, though unused in this example).

The data is stored in `.pkl` files and each file contains:
- `camera`: RGB image (shape: H x W x 3)
- `sdc_history_feature`: the past trajectory of the car
- `sdc_future_feature`: the future trajectory to predict
- `depth`: ground truth depth map (shape: H x W x 1)

We'll define a `DrivingDataset` class to load and return these tensors in a format our model can work with.

In [1]:
import torch
from torch.utils.data import Dataset
import pickle

class DrivingDataset(Dataset):
    def __init__(self, file_list, test=False):
        self.samples = file_list
        self.test = test

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        # Load pickle file
        with open(self.samples[idx], 'rb') as f:
            data = pickle.load(f)

        # Convert numpy arrays to tensors
        camera = torch.FloatTensor(data['camera']).permute(2, 0, 1)
        history = torch.FloatTensor(data['sdc_history_feature'])
        command = self.process_command(data['driving_command'])
        semantic = self.process_semantic(data['semantic_label'])
        depth = torch.FloatTensor(data['depth']).permute(2, 0, 1)

        if not self.test:
          future = torch.FloatTensor(data['sdc_future_feature'])
          return {
            'camera': camera,
            'history': history,
            'future': future,
            'command': command,
            'semantic': semantic,
            'depth': depth
          }
        else:
          return {
            'camera': camera,
            'history': history,
            'semantic': semantic,
            'command': command,
            'depth': depth
          }

    def process_semantic(self, label):
        num_classes = 63
        semantic_map = torch.zeros((num_classes, label.shape[0], label.shape[1]), dtype=torch.float32)
        for class_id in range(num_classes):
            semantic_map[class_id] = (torch.from_numpy(label) == class_id).float()
        lane_line_class = 12
        semantic_map[lane_line_class] *= 2.0  #
        return semantic_map

    def process_command(self, cmd_str):
        #
        cmd_map = {'forward': [1, 0, 0], 'left': [0, 1, 0], 'right': [0, 0, 1]}
        return torch.tensor(cmd_map[cmd_str], dtype=torch.float32)



Visualize the dataset

In [2]:
import pickle
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import random

k = 4
# load the data
data = []
for i in random.choices(np.arange(1000), k=k):
    with open(f"train/{i}.pkl", "rb") as f:
        data.append(pickle.load(f))

# plot the camera view of current step for the k examples
fig, axis = plt.subplots(1, k, figsize=(4*k, 4))
for i in range(k):
    axis[i].imshow(data[i]["camera"])
    axis[i].axis("off")
plt.suptitle("Camera view")
plt.tight_layout()
plt.show()

# plot the past and future trajectory of the vehicle
fig, axis = plt.subplots(1, k, figsize=(4*k, 4))
for i in range(k):
    axis[i].plot(data[i]["sdc_history_feature"][:, 0], data[i]["sdc_history_feature"][:, 1], "o-", color="gold", label="Past")
    axis[i].plot(data[i]["sdc_future_feature"][:, 0], data[i]["sdc_future_feature"][:, 1], "o-", color="green", label="Future")
    axis[i].legend()
    axis[i].axis("equal")
plt.suptitle("Past & future trajectories")
plt.tight_layout()
plt.show()

# plot feature space
semantic_colormap = {
    0: (0, 0, 0),         # UNLABELED
    1: (0, 0, 142),       # CAR
    2: (0, 0, 70),        # TRUCK
    3: (220, 20, 60),     # PEDESTRIAN
    4: (119, 11, 32),     # BIKE
    5: (152, 251, 152),   # TERRAIN
    6: (128, 64, 128),    # ROAD
    7: (244, 35, 232),    # SIDEWALK
    8: (70, 130, 180),    # SKY
    9: (250, 170, 30),    # TRAFFIC_LIGHT
    10: (190, 153, 153),  # FENCE
    11: (220, 220, 0),    # TRAFFIC_SIGN
    12: (255, 255, 255),  # LANE_LINE
    13: (55, 176, 189),   # CROSSWALK
    14: (0, 60, 100)      # BUS
}

fig, axis = plt.subplots(1, k, figsize=(4*k, 4))
for i in range(k):
    axis[i].imshow(data[i]["semantic_label"])
    axis[i].axis("off")
plt.suptitle("Semantic features")
plt.tight_layout()
plt.show()


fig, axis = plt.subplots(1, k, figsize=(4*k, 4))
ims = []
for i in range(k):
    im = axis[i].imshow(data[i]["depth"])
    axis[i].axis("off")
    ims.append(im)

plt.suptitle("Depth")
plt.tight_layout()
fig.colorbar(ims[-1], shrink=0.6)
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'train/974.pkl'

In [ ]:
class Logger:
    def __init__(self):
        # Placeholder for potential future configs (e.g., log_dir, wandb_enabled, etc.)
        self.train_loss = []
        self.val_loss = []
        self.ADE = []
        self.FDE = []
        self.MSE = []

    def log(self, step=None, **metrics):
        """
        Logs the given metrics.

        Args:
            step (int, optional): The current step or epoch. Useful for tracking.
            **metrics: Arbitrary keyword arguments representing metric names and values.
        """
        prefix = f"[Step {step}] " if step is not None else ""
        metric_str = " | ".join(f"{k}: {v}" for k, v in metrics.items())
        for k, v in metrics.items():
            if k == "train_loss":
                self.train_loss.append(v)
            elif k == "val_loss":
                self.val_loss.append(v)
            elif k == "ADE":
                self.ADE.append(v)
            elif k == "FDE":
                self.FDE.append(v)
            elif k == "MSE":
                self.MSE.append(v)
        # print(prefix + metric_str)

## 🧠 The Model: Trajectory + Depth Prediction

We've extended our trajectory prediction model to optionally include a **depth estimation decoder**.

Why?
- Predicting depth helps the model **learn richer visual features** from the camera input.
- This acts as a form of **multi-task learning**, where learning to estimate depth reinforces scene understanding, ultimately leading to better trajectory predictions.
- This can be especially useful in complex environments with occlusions or sharp turns.

The model has:
- A CNN backbone to extract features from the image
- An MLP to process historical trajectory features
- A trajectory decoder to predict future coordinates
- (Optionally) A depth decoder to predict dense depth maps

This auxiliary task is enabled by setting `use_depth_aux=True`.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DrivingPlanner(nn.Module):
    def __init__(self, use_depth_aux=False):
        super().__init__()
        self.use_depth_aux = use_depth_aux

        self.cnn_backbone = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2),
            nn.ReLU()
        )

        self.cnn_backbone = nn.Sequential(
            nn.Conv2d(3 , 32, 5, stride=2, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 5, stride=2, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 5, stride=2, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((8, 8)),
            nn.Dropout(0.3)  # Preventing Overfitting
        )

        self.cnn_flatten = nn.Flatten()

        self.history_encoder = nn.Sequential(
            nn.Linear(21 * 3, 128),
            nn.ReLU()
        )

        self.command_embedding = nn.Embedding(3, 32)

        self.fc = nn.Sequential(
            nn.Linear(8352, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 60 * 3)  # Predict the trajectory of the next 60 steps
        )

        # Optional depth decoder
        if self.use_depth_aux:
            self.depth_decoder = nn.Sequential(
                nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # Upsample
                nn.ReLU(),
                nn.Conv2d(64, 1, kernel_size=3, padding=1),
                nn.Upsample(size=(200, 300), mode='bilinear', align_corners=False)
            )

    def forward(self, camera, history, command):

        img_feat = self.cnn_backbone(self.normalize_camera(camera))           # [32, 128, 8, 8]
        # print(img_feat.shape)
        img_feat_flat = self.cnn_flatten(img_feat)
        hist_feat = self.history_encoder(history.view(history.size(0), -1))
        command_label = torch.argmax(command, dim=1)
        cmd_feat = self.command_embedding(command_label)

        feat = torch.cat([img_feat_flat, hist_feat, cmd_feat], dim=-1)
        future = self.fc(feat).view(-1, 60, 3)

        # Optional depth map prediction
        depth_out = None
        if self.use_depth_aux:
            depth_out = self.depth_decoder(img_feat.detach())
            # print(depth_out.shape)


        return future, depth_out

    def normalize_camera(self, img):
        # Normalized by ImageNet mean standard deviation
        mean = torch.tensor([0.485, 0.456, 0.406], device=img.device).view(1, 3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225], device=img.device).view(1, 3, 1, 1)
        return (img - mean) / std


## 🏋️ Training with Auxiliary Loss

The training loop is similar to Phase 1 — except now, if enabled, we also compute a loss on the predicted **depth map**.

We define:
- `trajectory_loss` as standard MSE between predicted and true future trajectory
- `depth_loss` as L1 loss between predicted and ground truth depth

Total loss = `trajectory_loss + lambda * depth_loss`

This helps guide the model to learn better representations from visual input. The weight `lambda` is a hyperparameter you can tune!

In [ ]:
import torch
import torch.nn.functional as F

def train_one_epoch(model, train_loader, optimizer, device, lambda_depth=0.1, use_depth_aux=False):
    model.train()
    criterion = nn.SmoothL1Loss()  #
    train_loss = 0.0

    for batch in train_loader:
        cam, hist, fut, depth, command = [batch[k].to(device) for k in ['camera', 'history', 'future', 'depth', 'command']]
        optimizer.zero_grad()
        fut_pred, dep_pred = model(cam, hist, command)

        traj_loss = F.mse_loss(fut_pred, fut)
        loss_heading = criterion(fut_pred[..., 2], fut[..., 2])
        loss = traj_loss + 0.2 * loss_heading

        if use_depth_aux:
            loss += lambda_depth * F.l1_loss(dep_pred, depth)

        is_turning = (command[:,1] == 1) | (command[:,2] == 1)  # left or right
        weights = torch.where(is_turning, 1.5, 1.0).to(device)
        loss = (loss * weights).mean()

        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_loss = train_loss / len(train_loader)
    return avg_loss

def validate(model, val_loader, device, lambda_depth=0.1, use_depth_aux=False):
    model.eval()
    total_ade, total_fde, total_mse, total_loss = 0.0, 0.0, 0.0, 0.0
    count = 0

    with torch.no_grad():
        for batch in val_loader:
            cam = batch['camera'].to(device)           # (batch, 3, H, W)
            semantic = batch['semantic'].to(device)    # (batch, 15, H, W)
            hist = batch['history'].to(device)         # (batch, 21, 3)
            command = batch['command'].to(device)      # (batch, 3)
            fut = batch['future'].to(device)           # (batch, 60, 3)
            depth = batch['depth'].to(device)

            fut_pred, dep_pred = model(cam, hist, command)

            B, T, _ = fut.shape
            count += B

            ade = torch.norm(fut_pred[:, :, :2] - fut[:, :, :2], dim=2).mean(dim=1).sum()
            fde = torch.norm(fut_pred[:, -1, :2] - fut[:, -1, :2], dim=1).sum()
            mse = F.mse_loss(fut_pred, fut, reduction='sum')

            if use_depth_aux:
                loss = mse + lambda_depth * F.l1_loss(dep_pred, depth)
                total_loss += loss.item()

            total_ade += ade.item()
            total_fde += fde.item()
            total_mse += mse.item()


    ade_avg = total_ade / count
    fde_avg = total_fde / count
    mse_avg = total_mse / (count * T * 3)


    loss_avg = total_loss / len(val_loader)
    return ade_avg, fde_avg, mse_avg, loss_avg

def train(model, train_loader, val_loader, optimizer, logger, num_epochs=50, lambda_depth=0.1, use_depth_aux=False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model = model.to(device)

    for epoch in range(num_epochs):
        train_loss = train_one_epoch(model, train_loader, optimizer, device, lambda_depth, use_depth_aux)
        ade, fde, mse, val_loss = validate(model, val_loader, device)

        if use_depth_aux:
            logger.log(step=epoch, train_loss=train_loss, val_loss=val_loss, ADE=ade, FDE=fde, MSE=mse)
            print(f"Epoch {epoch+1}, Loss: {train_loss:.3f}, Validation - ADE: {ade:.3f}, FDE: {fde:.3f}, Traj MSE: {mse:.3f}, Loss (MSE+depth): {val_loss:.3f}")
        else:
            logger.log(step=epoch, train_loss=train_loss, ADE=ade, FDE=fde, MSE=mse)
            print(f"Epoch {epoch+1}, Loss: {train_loss:.3f}, Validation - ADE: {ade:.3f}, FDE: {fde:.3f}, Traj MSE: {mse:.3f}")




In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader
import os

train_data_dir = "train"
val_data_dir = "val"

train_files = [os.path.join(train_data_dir, f) for f in os.listdir(train_data_dir) if f.endswith('.pkl')]
val_files = [os.path.join(val_data_dir, f) for f in os.listdir(val_data_dir) if f.endswith('.pkl')]

train_dataset = DrivingDataset(train_files)
val_dataset = DrivingDataset(val_files)

train_loader = DataLoader(train_dataset, batch_size=32, num_workers=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, num_workers=2)


## 🔍 Let's Compare Two Settings

We'll now train and evaluate the model in **two modes**:

1. **Without auxiliary task** — the model only predicts the trajectory.
2. **With depth auxiliary task** — the model also predicts a depth map, which helps it learn better visual features.

By comparing the results (ADE, FDE, and Trajectory MSE), you'll see the benefit of multi-task learning in action! 🚀

In [ ]:
use_depth_aux=False
model_no_aux = DrivingPlanner(use_depth_aux=use_depth_aux)
optimizer = optim.Adam(model_no_aux.parameters(), lr=1e-3)
logger_no_aux = Logger()
train(model_no_aux, train_loader, val_loader, optimizer, logger_no_aux, num_epochs=10,use_depth_aux=use_depth_aux)

Using device: cuda
Epoch 1, Loss: 22.9186, Validation - ADE: 4.0099, FDE: 9.3770, Traj MSE: 12.251686
Epoch 2, Loss: 12.1647, Validation - ADE: 3.3641, FDE: 8.3131, Traj MSE: 9.574406
Epoch 3, Loss: 9.8522, Validation - ADE: 3.4812, FDE: 8.5076, Traj MSE: 9.062673
Epoch 4, Loss: 7.8995, Validation - ADE: 2.9848, FDE: 7.4598, Traj MSE: 7.337304
Epoch 5, Loss: 6.8192, Validation - ADE: 2.4812, FDE: 6.5501, Traj MSE: 5.998971
Epoch 6, Loss: 6.2871, Validation - ADE: 2.4374, FDE: 6.6189, Traj MSE: 5.879153
Epoch 7, Loss: 6.1018, Validation - ADE: 2.7324, FDE: 7.0414, Traj MSE: 6.175226
Epoch 8, Loss: 5.8419, Validation - ADE: 2.9170, FDE: 7.3691, Traj MSE: 6.510932
Epoch 9, Loss: 5.6277, Validation - ADE: 2.4065, FDE: 6.4122, Traj MSE: 5.614890
Epoch 10, Loss: 5.3516, Validation - ADE: 2.4830, FDE: 6.5718, Traj MSE: 5.395532


In [ ]:
use_depth_aux=True
model_with_aux = DrivingPlanner(use_depth_aux=use_depth_aux)
optimizer = optim.Adam(model_with_aux.parameters(), lr=1e-3)
logger_aux = Logger()
train(model_with_aux, train_loader, val_loader, optimizer, logger_aux, num_epochs=50,use_depth_aux=use_depth_aux, lambda_depth=0.05)

Using device: cuda


ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
# plot 4 losses in 4 plots
fig, axis = plt.subplots(1, 4, figsize=(16, 4))
axis[0].plot(logger_no_aux.train_loss)
axis[0].set_title("Train Loss")
axis[1].plot(logger_no_aux.MSE)
axis[1].set_title("MSE")
axis[2].plot(logger_no_aux.ADE)
axis[2].set_title("ADE")
axis[3].plot(logger_no_aux.FDE)
axis[3].set_title("FDE")

plt.show()

In [ ]:
torch.save(model_with_aux.state_dict(), "phase2_model_w_depth.pth")
from google.colab import files
files.download("phase2_model_w_depth.pth")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ade, fde, mse = validate(model_no_aux, val_loader, device)
print(f"Validation results for model without depth auxiliary loss: ADE: {ade:.4f}, FDE: {fde:.4f}, Traj MSE: {mse:.6f}")

ade, fde, mse, val_loss = validate(model_with_aux, val_loader, device)
print(f"Validation results for model with depth auxiliary loss: ADE: {ade:.4f}, FDE: {fde:.4f}, Traj MSE: {mse:.6f}, Val. loss (depth): {mse:.6f}")

Validation results for model without depth auxiliary loss: ADE: 2.4830, FDE: 6.5718, Traj MSE: 5.395532


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv_transpose2d, but got input of size: [32, 8352]

## 🔍 Final Visualization and Comparison

Now that we’ve trained two models — one **with** the depth auxiliary task and one **without** — let’s visualize and compare their predictions.

We’ll show:
1. The **camera image** from selected validation examples
2. The **past trajectory**, **ground-truth future**, and **predicted future** trajectory
3. The **predicted vs. ground-truth depth maps** (only for the model trained with the auxiliary task)

These visualizations help us understand:
- Does the predicted trajectory better match the future when the depth task is included?
- Is the predicted depth map reasonably accurate?

Let’s see the difference! 📈

In [ ]:
from codeop import CommandCompiler
import matplotlib.pyplot as plt
import random

random.seed(40)

def visualize_comparison(val_loader, model_no_aux, model_with_aux, device):
    model_no_aux.eval()
    model_with_aux.eval()
    val_batch = next(iter(val_loader))

    camera = val_batch['camera'].to(device)
    history = val_batch['history'].to(device)
    future = val_batch['future'].to(device)
    depth = val_batch['depth'].to(device)
    command = val_batch['command'].to(device)

    with torch.no_grad():
        pred_no_aux, _ = model_no_aux(camera, history, command)
        pred_with_aux, pred_depth = model_with_aux(camera, history, command)

    camera = camera.cpu().numpy()
    history = history.cpu().numpy()
    future = future.cpu().numpy()
    pred_no_aux = pred_no_aux.cpu().numpy()
    pred_with_aux = pred_with_aux.cpu().numpy()
    depth = depth.cpu().numpy()
    pred_depth = pred_depth.cpu().numpy() if pred_depth is not None else None

    k = 4
    indices = random.choices(np.arange(len(camera)), k=k)

    # Show the input camera images
    fig, ax = plt.subplots(1, k, figsize=(4 * k, 4))
    for i, idx in enumerate(indices):
        ax[i].imshow(camera[idx].transpose(1, 2, 0))
        ax[i].set_title(f"Example {i+1}")
        ax[i].axis("off")
    plt.suptitle("Camera Inputs")
    plt.tight_layout()
    plt.show()

    # Compare predicted trajectories
    fig, ax = plt.subplots(2, k, figsize=(4 * k, 8))
    for i, idx in enumerate(indices):
        # Without aux
        ax[0, i].plot(history[idx, :, 0], history[idx, :, 1], 'o-', label='Past', color='gold', markersize=4, linewidth=1.2)
        ax[0, i].plot(future[idx, :, 0], future[idx, :, 1], 'o-', label='GT Future', color='green', markersize=4, linewidth=1.2)
        ax[0, i].plot(pred_no_aux[idx, :, 0], pred_no_aux[idx, :, 1], 'o-', label='Pred (No Aux)', color='red', markersize=4, linewidth=1.2)
        ax[0, i].set_title("No Depth Aux")
        ax[0, i].axis("equal")

        # With aux
        ax[1, i].plot(history[idx, :, 0], history[idx, :, 1], 'o-', label='Past', color='gold', markersize=4, linewidth=1.2)
        ax[1, i].plot(future[idx, :, 0], future[idx, :, 1], 'o-', label='GT Future', color='green', markersize=4, linewidth=1.2)
        ax[1, i].plot(pred_with_aux[idx, :, 0], pred_with_aux[idx, :, 1], 'o-', label='Pred (With Aux)', color='blue', markersize=4, linewidth=1.2)
        ax[1, i].set_title("With Depth Aux")
        ax[1, i].axis("equal")

    # Show full legend in a new figure
    fig_legend = plt.figure(figsize=(8, 1))
    legend_handles = [
        plt.Line2D([0], [0], color='gold', marker='o', linestyle='-', markersize=5, label='Past'),
        plt.Line2D([0], [0], color='green', marker='o', linestyle='-', markersize=5, label='GT Future'),
        plt.Line2D([0], [0], color='red', marker='o', linestyle='-', markersize=5, label='Pred (No Aux)'),
        plt.Line2D([0], [0], color='blue', marker='o', linestyle='-', markersize=5, label='Pred (With Aux)')
    ]
    fig_legend.legend(handles=legend_handles, loc='center', ncol=4)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    plt.suptitle("Trajectory Prediction: Without vs With Depth Aux Task")
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

    # Show predicted vs GT depth (only for bottom row)
    if pred_depth is not None:
        fig, ax = plt.subplots(2, k, figsize=(4 * k, 6))
        for i, idx in enumerate(indices):
            ax[0, i].imshow(depth[idx, :, :, 0], cmap='viridis')
            ax[0, i].set_title("GT Depth", pad=10)
            ax[0, i].axis("off")
            # increase vertical distance between rows

            ax[1, i].imshow(pred_depth[idx, :, :, 0], cmap='viridis')
            ax[1, i].set_title("Pred Depth", pad=10)
            ax[1, i].axis("off")

        plt.suptitle("Depth Estimation (Only for Model With Aux Task)", y=1.05)
        plt.subplots_adjust(hspace=0.4)
        plt.tight_layout()
        plt.show()


# 🔚 Call at the end after training both models
visualize_comparison(val_loader, model_no_aux, model_with_aux, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

<built-in method size of Tensor object at 0x7e887ff0e270>


RuntimeError: Given transposed=1, weight of size [32, 16, 4, 4], expected input[32, 128, 8, 8] to have 32 channels, but got 128 channels instead

Now we run our model on the test set once, to get the plan of our model and save it for submission. Notice that the ground truth plans are removed for the test set, so you can not calculate the ADE metric on the test set yourself, and need to submit it to the leader board. By running the last cell, you'll be able to see a csv file called submission_phase2.csv by clicking on the folder icon on the left. Download it and submit it to the leaderboard to get your score.

In [ ]:
with open(f"test_public/0.pkl", "rb") as f:
    data = pickle.load(f)
print(data.keys())
# Note the absence of sdc_future_feature

dict_keys(['camera', 'depth', 'driving_command', 'sdc_history_feature', 'semantic_label'])


In [ ]:
import pandas as pd
test_data_dir = "test_public"
test_files = [os.path.join(test_data_dir, fn) for fn in sorted([f for f in os.listdir(test_data_dir) if f.endswith(".pkl")], key=lambda fn: int(os.path.splitext(fn)[0]))]
test_dataset = DrivingDataset(test_files, test=True)
test_loader = DataLoader(test_dataset, batch_size=250, num_workers=2)
model_with_aux.eval()
all_plans = []
with torch.no_grad():
    for batch in test_loader:
        camera = batch['camera'].to(device)
        history = batch['history'].to(device)

        pred_future, _ = model_with_aux(camera, history)
        all_plans.append(pred_future.cpu().numpy()[..., :2])
all_plans = np.concatenate(all_plans, axis=0)

# Now save the plans as a csv file
pred_xy = all_plans[..., :2]  # shape: (total_samples, T, 2)

# Flatten to (total_samples, T*2)
total_samples, T, D = pred_xy.shape
pred_xy_flat = pred_xy.reshape(total_samples, T * D)

# Build a DataFrame with an ID column
ids = np.arange(total_samples)
df_xy = pd.DataFrame(pred_xy_flat)
df_xy.insert(0, "id", ids)

# Column names: id, x_1, y_1, x_2, y_2, ..., x_T, y_T
new_col_names = ["id"]
for t in range(1, T + 1):
    new_col_names.append(f"x_{t}")
    new_col_names.append(f"y_{t}")
df_xy.columns = new_col_names

# Save to CSV
df_xy.to_csv("submission_phase2.csv", index=False)

print(f"Shape of df_xy: {df_xy.shape}")

Shape of df_xy: (1000, 121)
